In [519]:
import pandas as pd
import numpy as np
import math
import xgboost as xgb
import os
import random
import tensorflow as tf
import sklearn.metrics as metrics


from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from arch import arch_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

In [520]:
# Set a seed value
seed_value= 40
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

data = pd.read_csv('features.csv')

data.drop_duplicates(subset='Date', keep='last', inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values('Date', inplace=True)
data.set_index('Date', inplace=True)

data = data.asfreq('D') 
data.fillna(method='ffill', inplace=True) 

data['DCOILBRENTEU'] = data['DCOILBRENTEU'].replace('.', np.nan)
data['DCOILBRENTEU'] = data['DCOILBRENTEU'].ffill()
data['DCOILBRENTEU'] = data['DCOILBRENTEU'].astype(float)
data['DCOILBRENTEU_ret'] = data['DCOILBRENTEU'].pct_change()

data['DJIA_ret'] = data['Dow Jones Industrial Average Close'].pct_change()

data['daily_vol_t+1'] = data['daily_vol'].shift(-1)
data = data.drop(data.index[-1])
data = data.drop(data.index[0])

missing_values_count = data.isna().sum()
columns_with_missing_values = missing_values_count[missing_values_count > 0].index.tolist()
columns_with_full_data = missing_values_count[missing_values_count == 0].index.tolist()

print("Columns with missing values:", columns_with_missing_values,'\n')
print("Columns with full values:")

for col in columns_with_full_data:
    print(col)

Columns with missing values: ['count', 'sentiment', 'news_count_after_close', 'bearish', 'bullish', 'neutral'] 

Columns with full values:
Unnamed: 0
last_last_hour_ret
last_daily_ret
last_last_hour_whole_day_ret_ratio
last_skewness_last_hour
last_kurtosis_last_hour
last_skewness_day
last_kurtosis_day
last_daily_ret_to_mean_5day_ret
last_skew_ret
last_kurt_ret
last_daily_ret_lag1
last_daily_ret_lag2
last_daily_ret_lag3
last_daily_ret_lag4
last_daily_ret_lag5
high_last_hour_ret
high_daily_ret
high_last_hour_whole_day_ret_ratio
high_skewness_last_hour
high_kurtosis_last_hour
high_skewness_day
high_kurtosis_day
high_daily_ret_to_mean_5day_ret
high_skew_ret
high_kurt_ret
high_daily_ret_lag1
high_daily_ret_lag2
high_daily_ret_lag3
high_daily_ret_lag4
high_daily_ret_lag5
mean_last_hour_ret
mean_daily_ret
mean_last_hour_whole_day_ret_ratio
mean_skewness_last_hour
mean_kurtosis_last_hour
mean_skewness_day
mean_kurtosis_day
mean_daily_ret_to_mean_5day_ret
mean_skew_ret
mean_kurt_ret
mean_daily_

In [521]:
# Replace these with your actual feature column names
target_feature = ['daily_vol_t+1']
vol_features = ['daily_vol','daily_vol_lag1','daily_vol_lag2','daily_vol_lag3','last_hour_vol']
ret_features = [ 
    'last_daily_ret',
    'last_last_hour_ret',
    'low_daily_ret',
    'high_last_hour_ret',
    'high_daily_ret',
    'last_daily_ret_lag1',
    'last_daily_ret_lag2',
    'high_daily_ret_lag1',
    'high_daily_ret_lag2',
    'mean_daily_ret_lag1',
    'mean_daily_ret_lag2']


trading_features = ['Volume','10Y','1Y','4W']

Industry_features = ['DCOILBRENTEU_ret','eps','DJIA_ret']

sentiment_features = ['VIX_open','VIX_high','VIX_low','VIX_close','news_count_after_close_polygon']

Econ_features = ['CPI','FEDFUNDS','CPI_12_month_pct']

features = vol_features + ret_features + trading_features + Industry_features + sentiment_features + Econ_features

X = data[features+target_feature]
y = y = X.pop("daily_vol_t+1")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [522]:
# Scale the feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [523]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=len(X_train[0]), activation='relu'))
model.add(Dropout(0.02)) 
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.05)) 
model.add(Dense(64, activation='relu')) 
model.add(Dense(1, activation='linear'))

# Compile the model with mean squared error loss and the Adam optimizer with a lower learning rate
model.compile(loss='mean_squared_error', optimizer='adam')

# Add early stopping to prevent overfitting
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

# Fit the model
history = model.fit(
    X_train,
    y_train,
    epochs=300,
    validation_data=(X_test, y_test),
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping_monitor]
)

# Predict the test_y
y_test_pred = model.predict(X_test)
rmse_test = sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Test RMSE: {rmse_test}')

#Flatten the array into 1-D
y_test = y_test.ravel()
y_test_pred = y_test_pred.ravel()

r_squared = r2_score(y_test, y_test_pred)
print(f"R-squared: {r_squared}")

Epoch 1/300
28/28 [==============================] - 1s 8ms/step - loss: 0.0345 - val_loss: 0.0050
Epoch 2/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0040 - val_loss: 0.0017
Epoch 3/300
28/28 [==============================] - 0s 8ms/step - loss: 0.0025 - val_loss: 0.0016
Epoch 4/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0021 - val_loss: 0.0010
Epoch 5/300
28/28 [==============================] - 0s 4ms/step - loss: 0.0014 - val_loss: 7.4467e-04
Epoch 6/300
28/28 [==============================] - 0s 4ms/step - loss: 0.0014 - val_loss: 6.3829e-04
Epoch 7/300
28/28 [==============================] - 0s 3ms/step - loss: 0.0013 - val_loss: 6.4589e-04
Epoch 8/300
28/28 [==============================] - 0s 4ms/step - loss: 9.5517e-04 - val_loss: 6.1214e-04
Epoch 9/300
28/28 [==============================] - 0s 4ms/step - loss: 8.3336e-04 - val_loss: 4.5411e-04
Epoch 10/300
28/28 [==============================] - 0s 4ms/step - loss: 6.3071e

In [524]:
def evaluate(y_test, y_test_pred):
    """
    Function to calculate RMSE for one-dimensional arrays
    """
    mse = np.mean((y_test - y_test_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

rmse = evaluate(y_test, y_test_pred)

MAX_POSSIBLE_RMSE = y_test.std()
normalized_rmse = (1 - rmse / MAX_POSSIBLE_RMSE)

Grade = normalized_rmse
print('The grade for this assignment is ', math.ceil(Grade * 100))

The grade for this assignment is  50
